title: Premium value reminders (atm fair use)   
author: Fabio Schmidt-Fischbach   
date: 2020-11-11   
region: EU   
summary: Contextual info cards that are triggered after ATM withdrawals. Set of ATM upgrade triggers targeted at non-premium users (upsell triggers) + ATM value reminders targeted at premium users. 95% of existing users received the treatment - 5% did not. Goal was to increase awareness of ATM/Fair-use and premium value. Only the infocard that fires after every withdrawal has relevant reach (20% of MAUs saw it at least once). Due to the fairly limited reach, the # of upgrades attributed to the infocards are not large (10-40 each week). 5% of infocards are clicked on - 20% of them are dismissed each week. % of users that visited the ATM screen is 2pp higher in variant vs control.No clear effect on CS contacts, ATM withdrawals, Premium retention (a bit early to call this). 
link: https://docs.google.com/presentation/d/1bik1Zk6Irfw-EDDCNVY1uvsBWxJasYr3Y9lVMFVQ8ls/edit?usp=sharing   
tags: premium, value reminder, membership, infocards, atm, fair use

In [2]:
import altair as alt
import pandas as pd
import numpy as np
from datetime import date

In [27]:
query = """
select user_id, 
	is_user_in_tg(user_id, 'USER-ID:',95) as segment,
	collector_tstamp, 
	mt.name, 
	case when event_type = -894 then 'view' when event_type = 6257 and se_label = 'cta1' then 'click' else 'dismiss' end as event_type 
from dbt.snowplow as sn 
inner join mcv_infocard mcv on mcv.id = sn.se_property 
inner join mcv_infocard_template mt 
    on mt.id = mcv.template_id 
    and ( mt.name like 'MEM_VALUEREMINDER%' or  mt.name like 'MEM_UPGRADE%' ) 
where event_type in (-894, 6257) and collector_date >= '2020-08-01'

"""

df = pd.read_csv("infocards.csv")

# drop current week
df = df.loc[df["collector_tstamp"] < "2020-11-01", :]

df["date"] = pd.to_datetime(df["collector_tstamp"]).dt.week.astype(str)

df = df.groupby(["date", "segment", "name"])["user_id"].agg("nunique").reset_index()

df["name"] = df["name"].str.replace("MEM_UPGRADE_", "")
df["name"] = df["name"].str.replace("MEM_VALUEREMINDER_", "")


alt.Chart(df).mark_line().encode(
    x="date:N",
    y=alt.Y("user_id:Q", axis=alt.Axis(title="Number of users")),
    color="name:N",
).properties(width=500, height=500, title="Reach of infocards in total")

alt.Chart(...)

In [38]:
active_users = """
select dwh.week, 
	zu.user_id, 
	is_user_in_tg(user_id, 'USER-ID:',95) as segment,
	zup.product_id, 
	zup.market, 
	max(case when act.user_created is null then 0 else 1 end) as mau, 
	max(case when txn.user_created is null then 0 else 1 end) as atm,
	sum(n_card_atm) as atm 
from dwh_cohort_weeks as dwh 
left join dbt.zrh_user_activity_txn as act 
	on act.activity_type = '1_tx_35' 
	and end_time between activity_start and activity_end 
left join dbt.zrh_user_product as zup 
	on zup.user_created = act.user_created 
	and end_time between subscription_valid_from and subscription_valid_until 
inner join dbt.zrh_users as zu on zu.user_created = act.user_created 
left join dbt.zrh_txn_day as txn 
	on txn.user_created = zu.user_created 
	and txn.txn_date::date between start_time and end_time 
	and n_card_atm > 0 
where dwh.start_time >= '2020-09-20' and dwh.start_time < current_date
group by 1,2,3,4,5
"""

df = pd.read_csv("infocards.csv")

df["name"] = df["name"].str.replace("MEM_UPGRADE_", "")
df["name"] = df["name"].str.replace("MEM_VALUEREMINDER_", "")
# drop current week
df = df.loc[df["collector_tstamp"] < "2020-11-01", :]
df["date"] = pd.to_datetime(df["collector_tstamp"]).dt.week.astype(str)
df["date"] = "2020-" + df["date"]

# pivot
df["x"] = 1
df = (
    pd.pivot_table(
        df.loc[:, ["user_id", "date", "x", "name"]],
        columns=["name"],
        index=["user_id", "date"],
        aggfunc=np.sum,
    )
    .fillna(0)
    .reset_index()
)
df.columns = df.columns.map("|".join).str.strip("|")
# merge to activity table.
active = pd.read_csv("active.csv")

active = active.merge(
    df, left_on=["week", "user_id"], right_on=["date", "user_id"], how="left"
).fillna(0)

active = active.loc[active["product_id"] != 0, :]
active = active.loc[active["product_id"] != "FLEX_ACCOUNT_MONTHLY", :]
active = active.loc[active["mau"] == 1, :]
active = active.loc[active["segment"] == True, :]

active = active.drop(columns=["segment", "market", "mau", "atm", "atm.1", "date"])

# make it long again.
active = pd.melt(active, id_vars=["user_id", "product_id", "week"]).reset_index()

# replace values > 1 by 1
active.loc[active["value"] > 1, "value"] = 1

active = (
    active.groupby(["week", "product_id", "variable"])["value"]
    .agg("mean")
    .reset_index()
)
active["variable"] = active["variable"].str.replace("x|", "")
active["variable"] = active["variable"].str.replace("|", "")

alt.Chart(active).mark_line().encode(
    x=alt.X("week:N", axis=alt.Axis(title="Week")),
    y=alt.Y("value:Q", axis=alt.Axis(format="%", title="% of MAUs")),
    color="variable:N",
).properties(width=300, height=300).facet(facet="product_id:N", columns=3)

alt.FacetChart(...)

In [40]:
df = pd.read_csv("infocards.csv")

df["name"] = df["name"].str.replace("MEM_UPGRADE_", "")
df["name"] = df["name"].str.replace("MEM_VALUEREMINDER_", "")

df["x"] = 1
df = df.groupby(["user_id", "name"])["x"].agg("sum").reset_index()

df = (
    pd.pivot_table(
        df.loc[:, ["user_id", "x", "name"]], columns=["name"], index=["user_id"]
    )
    .fillna(0)
    .reset_index()
)
df.columns = df.columns.map("".join).str.strip("")
df.columns = df.columns.str.replace("x", "")
# merge to activity table.
active = pd.read_csv("active.csv")

# only keep those that are currently MAUs
active = active.loc[(active["mau"] == 1) & (active["week"] == "2020-44"), :]
active = active.loc[active["product_id"] != "FLEX_ACCOUNT_MONTHLY", :]
active = active.loc[active["segment"] == True, :]
active = active.drop(columns=["segment", "market", "mau", "atm", "atm.1", "week"])

active = active.merge(df, left_on=["user_id"], right_on=["user_id"], how="left").fillna(
    0
)
active = active.loc[active["product_id"] != 0, :]

# make it long again.
active = pd.melt(active, id_vars=["user_id", "product_id"]).reset_index()

# replace values > 1 by 1
active.loc[active["value"] > 1, "value"] = 1

active = active.groupby(["product_id", "variable"])["value"].agg("mean").reset_index()


alt.Chart(
    active.loc[active["product_id"].isin(["STANDARD", "BUSINESS_CARD"]) == False, :]
).mark_bar().encode(
    x=alt.X("variable:N", axis=alt.Axis(title="Infocard")),
    y=alt.Y("value:Q", axis=alt.Axis(format="%", title="% of MAUs")),
).properties(
    width=500, height=300
).facet(
    facet="product_id:N", columns=2
)

alt.FacetChart(...)

In [54]:
df = pd.read_csv("infocards.csv")
df["name"] = df["name"].str.replace("MEM_UPGRADE_", "")
df["name"] = df["name"].str.replace("MEM_VALUEREMINDER_", "")

# drop current week
df = df.loc[df["collector_tstamp"] < "2020-11-01", :]

df["date"] = pd.to_datetime(df["collector_tstamp"]).dt.week.astype(str)
df = df.drop(columns=["collector_tstamp", "segment"])

df["x"] = 1
df = (
    pd.pivot_table(
        df,
        index=["user_id", "date", "name"],
        columns=["event_type"],
        values=["x"],
        aggfunc=np.sum,
    )
    .fillna(0)
    .reset_index()
)
df.columns = df.columns.map("".join).str.strip("")
df.columns = df.columns.str.replace("x", "")

df["perc"] = 100 * df["click"] / df["view"]
df.loc[df["view"] == 0, "perc"] = np.nan

df = df.groupby(["date", "name"])["perc"].agg("mean").reset_index()
df["date"] = "2020-" + df["date"]
alt.Chart(df).mark_line().encode(
    x=alt.X("date:N", axis=alt.Axis(title="Week")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="% of views that are clicked")),
    color="name",
).properties(width=500, height=500, title="% of viewed infocards that are clicked")

alt.Chart(...)

In [55]:
df = pd.read_csv("infocards.csv")
df["name"] = df["name"].str.replace("MEM_UPGRADE_", "")
df["name"] = df["name"].str.replace("MEM_VALUEREMINDER_", "")

# drop current week
df = df.loc[df["collector_tstamp"] < "2020-11-01", :]

df["date"] = pd.to_datetime(df["collector_tstamp"]).dt.week.astype(str)
df = df.drop(columns=["collector_tstamp", "segment"])

df["x"] = 1
df = (
    pd.pivot_table(
        df,
        index=["user_id", "date", "name"],
        columns=["event_type"],
        values=["x"],
        aggfunc=np.sum,
    )
    .fillna(0)
    .reset_index()
)
df.columns = df.columns.map("".join).str.strip("")
df.columns = df.columns.str.replace("x", "")

df["perc"] = 100 * df["dismiss"] / df["view"]
df.loc[df["view"] == 0, "perc"] = np.nan

df = df.groupby(["date", "name"])["perc"].agg("mean").reset_index()
df["date"] = "2020-" + df["date"]
alt.Chart(df).mark_line().encode(
    x=alt.X("date:N", axis=alt.Axis(title="Week")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="% of views that are dismissed")),
    color="name",
).properties(width=500, height=500, title="% of viewed infocards that are dismissed")

alt.Chart(...)

## How did users respond to the infocards? 

In [ ]:
df = pd.read_csv("infocards.csv")

df["name"] = df["name"].str.replace("MEM_UPGRADE_", "")
df["name"] = df["name"].str.replace("MEM_VALUEREMINDER_", "")

df["x"] = 1
df = df.groupby(["user_id", "name"])["x"].agg("sum").reset_index()

df = (
    pd.pivot_table(
        df.loc[:, ["user_id", "x", "name"]], columns=["name"], index=["user_id"]
    )
    .fillna(0)
    .reset_index()
)
df.columns = df.columns.map("".join).str.strip("")
df.columns = df.columns.str.replace("x", "")
# merge to activity table.
active = pd.read_csv("active.csv")

# keep only those that are MAUs and did an ATM in a week

# only keep those that are currently MAUs
active = active.loc[(active["mau"] == 1) & (active["week"] == "2020-44"), :]
active = active.loc[active["product_id"] != "FLEX_ACCOUNT_MONTHLY", :]
active = active.loc[active["segment"] == True, :]
active = active.drop(columns=["segment", "market", "mau", "atm", "atm.1", "week"])

active = active.merge(df, left_on=["user_id"], right_on=["user_id"], how="left").fillna(
    0
)
active = active.loc[active["product_id"] != 0, :]

# make it long again.
active = pd.melt(active, id_vars=["user_id", "product_id"]).reset_index()

# replace values > 1 by 1
active.loc[active["value"] > 1, "value"] = 1

active = active.groupby(["product_id", "variable"])["value"].agg("mean").reset_index()

alt.Chart(
    active.loc[active["product_id"].isin(["STANDARD", "BUSINESS_CARD"]) == False, :]
).mark_bar().encode(
    x=alt.X("variable:N", axis=alt.Axis(title="Infocard")),
    y=alt.Y("value:Q", axis=alt.Axis(format="%", title="% of MAUs")),
).properties(
    width=500, height=300
).facet(
    facet="product_id:N", columns=2
)

In [70]:
# merge to activity table.
active = pd.read_csv("premium_active.csv")
active = active.loc[
    (active["mau"] == 1)
    & (
        active["product_id"].isin(["STANDARD", "BUSINESS_CARD", "FLEX_ACCOUNT_MONTHLY"])
        == False
    ),
    :,
]
active = active.groupby(["segment", "week"])["atm"].agg("mean").reset_index()

alt.Chart(active).mark_line().encode(
    x=alt.X("week:N", axis=alt.Axis(title="Week")),
    y=alt.Y("atm:Q", axis=alt.Axis(format="%", title="% of MAUs with ATM withdrawals")),
    color="segment",
).properties(
    width=500, height=500, title="% of premium MAUs that did an ATM transaction"
)

alt.Chart(...)

In [73]:
cs = """
select user_id, contact_date, cs_tag 
from dbt.sf_all_contacts 
where cs_tag in ('atm/fair_use', 'metal', 'you', 'flex','business_you', 'business_metal') 
and contact_date >= '2020-08-01' and c_level_report = true  

"""
# merge to activity table.
active = pd.read_csv("premium_active.csv")
active = active.loc[
    (active["atm"] == 1)
    & (
        active["product_id"].isin(["STANDARD", "BUSINESS_CARD", "FLEX_ACCOUNT_MONTHLY"])
        == False
    ),
    :,
]

df = pd.read_csv("cs.csv")

# df = df.loc[df["cs_tag"]=='atm/fair_use',:]
df["week"] = "2020-" + pd.to_datetime(df["contact_date"]).dt.week.astype(str)
df = df.groupby(["week", "user_id"])["cs_tag"].agg("count").reset_index()

active = active.merge(df, on=["week", "user_id"], how="left")
active["cs_tag"] = active["cs_tag"].fillna(0)
active.loc[active["cs_tag"] > 1, "cs_tag"] = 1

active = active.groupby(["week", "segment"])["cs_tag"].agg("mean").reset_index()


alt.Chart(active).mark_line().encode(
    x=alt.X("week:N", axis=alt.Axis(title="Week")),
    y=alt.Y(
        "cs_tag:Q",
        axis=alt.Axis(
            format="%", title="% of premium ATM users with an ATM CS contact"
        ),
    ),
    color="segment",
).properties(
    width=500, height=500, title="% of premium ATM users with an ATM CS contact"
)

alt.Chart(...)

In [71]:
os.listdir()

['cs.csv',
 'active.csv',
 'infocards.csv',
 'premium_value_reminders_20201102.ipynb',
 'premium_active.csv',
 'active_old.csv',
 '.ipynb_checkpoints']